# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

- Dislplay the generated `tailored_resume.md` file.

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is an accomplished Full Stack Engineer with 18 years of experience in the tech industry, specializing in both front-end and back-end software development. He is proficient in multiple programming languages including Python, JavaScript, and Ruby, and has extensive experience with front-end frameworks such as React and Angular, as well as back-end development in Node.js. Noah has demonstrated expertise in building scalable, well-functioning applications and databases, making him a perfect fit for AI Fund's dynamic and innovative environment.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Led the development of scalable software solutions, incorporating AI technologies into the core product offerings.
- Managed full software development lifecycle, enhancing team's capabilities in building responsive and efficient applications.
- Played a key role in setting up robust security and data protection frameworks.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Orchestrated the strategic direction of software development, focusing on the integration of front-end and back-end systems to streamline operations.
- Enhanced database management and API development, boosting system responsiveness and user engagement.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Founded and led a tech startup from concept through execution, focusing on IoT solutions with a strong emphasis on database and user interface design.
- Developed the front-end and back-end of a GPS tracking system using MongoDB and React, demonstrating a hands-on approach to full-stack development.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Directed a team of engineers in developing software solutions, heavily utilizing JavaScript, Ruby on Rails, and React. Implemented numerous features that improved user interaction and data processing.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led software development projects that significantly improved business processes and customer interaction through enhanced software solutions.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Focused on optimizing a central API and improving data handling capabilities, critical for robust software development.

### BetCraft: CTO — 2013 - 2015
- Managed the full lifecycle of software development, from conception to deployment, specializing in using React and Angular to deliver compelling product features.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah's extensive experience in full-stack development, coupled with his leadership in innovative technology projects and his proficiency in the latest programming languages and frameworks, makes him an ideal candidate for the Full Stack Engineer position at AI Fund.

- Dislplay the generated `interview_materials.md` file.

In [18]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Noah Williams - Full Stack Engineer at AI Fund**

1. **Technical Expertise in Full Stack Development**
   - *Question:* Could you walk us through your experience with front-end and back-end development featured in projects like InnovPet and your role at DataKernel?
   - *Talking Point:* Highlight your hands-on experience with JavaScript, Ruby, and Python, and discuss your approach to selecting frameworks for various projects.

2. **Leadership in Software Engineering**
   - *Question:* As a Director at DataKernel, you've managed cross-functional remote teams. How do you ensure effective collaboration and communication within your team?
   - *Talking Point:* Discuss your leadership style and how it has evolved to manage high-performing remote teams effectively.

3. **Integration of AI Technologies**
   - *Question:* You’ve implemented AI technologies at DataKernel. Can you describe the process and the impact of these technologies on the business outcomes?
   - *Talking Point:* Focus on your strategic role in integrating AI and how it aligns with AI Fund’s objectives to innovate within its portfolio companies.

4. **Project Management and Delivery**
   - *Question:* Can you provide an example of a complex project you delivered from conception to deployment?
   - *Talking Point:* Use examples from your work history where you managed the full software development lifecycle, emphasizing time management, problem-solving, and technical skills.

5. **Building Scalable Solutions**
   - *Question:* What strategies do you employ to ensure the scalability and security of your software solutions?
   - *Talking Point:* Discuss your experience with databases and security frameworks, possibly touching on your work setting up robust data protection frameworks at DataKernel.

6. **Technical Problem Solving**
   - *Question:* Could you describe a challenging technical problem you encountered and how you addressed it?
   - *Talking Point:* Share specific technical challenges, particularly those involving backend optimizations or front-end usability improvements.

7. **Cultural Fit and Personal Motivation**
   - *Question:* AI Fund values accountability, grit, and an ownership orientation. Can you discuss how these values resonate with your professional experience and personal ethics?
   - *Talking Point:* Reflect on your personal commitment to these values through your entrepreneurial ventures and leadership roles.

8. **Future Vision and Innovation**
   - *Question:* Where do you see the biggest opportunity for technological advancement in AI Fund’s portfolio companies, and how would you contribute to it?
   - *Talking Point:* Leverage your background in AI and data science to suggest innovative approaches and technologies that could benefit AI Fund.

These questions and talking points are designed to help Noah showcase his technical capabilities, leadership qualities, and alignment with the culture and goals of AI Fund. They will also provide the interviewers with insights into his problem-solving skills and his potential to drive future innovations at the company.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)